In [2]:
import pandas as pd
import numpy as np
import re

### Summary of Data

- Data contains record of audiobooks from 1998 till 2025. Audiobooks of 2025 not released yet. Data has 8 columns containing information of different audiobook in each row. Information include Name of book its writer, narrator, time of audio book recording, release date, language of book content, rating to book with stars & price of book. Our goal is to make dataset which will provide information related to great book data has 87489 rows and 8 columns

### Column Breakdown
- name: Name of the audiobook
- author: Author of the audiobook
- narrator: Narrator of the audiobook
- time: Length of the audiobook
- releasedate: Release date of the audiobook
- language: Language of the audiobook
- stars: No. of stars the audiobook received
- price: Price of the audiobook in INR
- ratings: No. of reviews received by the audiobook

# Issues with Dataset 

### Dirty data issues (completeness,validity,accuracy,consistency)

#### name,author,narrator column
- name,author,narrator columns has encoding character issue, some characters are not encoded properly (while reading in pandas this issue handled UTF-8 encoding) (validity) &#10003;
- name,author,narrator columns contains different language data (consistency) &#10003;
- in author & narrator column first name and last name not seperated by white space(validity)
- No need of Writtenby: in author column (validity) &#10003;
- No need of Narratedby in narrator column (validity) &#10003;
- row 35 in names column book name contains book categories (14 Short Stories Bundle for Kids: Adventure, Science Fiction, Aesopâ€™s Fables, Classic Fairy Tales and Christmas Story Books for Children)(validity)
- There are no proper names in narrator, e.g., "Narratedby:uncredited".(accuracy)

#### releasedate
- need datatype change in releasedate column(validity) &#10003;

#### language
- in laguage some values are in title format and some in lowercase(consistency) &#10003;
- need datatype change in laguage column to category(validity) &#10003;

#### stars
- Some empty values are represented as "Not rated yet" in stars column. For that, we can assume total ratings = 0 and avg. ratings = 0.(completeness) &#10003; 

#### price
- price columns some values in float and some in int need dtype change(consistency) &#10003;
- Free mentioned in price replace with 0(completeness)  &#10003;

#### Suggestion
- if data has genre column it will help in book categorization &#128161;

### Messy data issues (Untidy data-Structural issue) 

#### name,author,narrator column
- name columns contains 2 info combined book name and its edition
- books written by multiple people Writtenby:EdwardWyke-Smith,VeronicaCossanteli comma seperated data
- books Narrated by multiple people comma seperated data

#### time
- in time column hrs and min in single column seperated by and need single column in minutes convert into int dtype &#10003;

#### stars
- stars has ratings and reviews in same columns need seperate column also rating is out of 5 no need to mention &#10003;

In [3]:
# with chunksize we are reading only first 1000 lines
audio = pd.read_csv(r"D:\Jupyter\audible_uncleaned(in).csv",chunksize=1000)
for i in audio:
    df1 = i.copy()
    break

In [4]:
# check for data structure with first 5 values 
df1.head()

,name,author,narrator,time,releasedate,language,stars,price
0,Geronimo Stilton #11 & #12,Writtenby:GeronimoStilton,Narratedby:BillLobely,2 hrs and 20 mins,08-04-2008,English,5 out of 5 stars34 ratings,468
1,The Burning Maze,Writtenby:RickRiordan,Narratedby:RobbieDaymond,13 hrs and 8 mins,05-01-2018,English,4.5 out of 5 stars41 ratings,820
2,The Deep End,Writtenby:JeffKinney,Narratedby:DanRussell,2 hrs and 3 mins,11-06-2020,English,4.5 out of 5 stars38 ratings,410
3,Daughter of the Deep,Writtenby:RickRiordan,Narratedby:SoneelaNankani,11 hrs and 16 mins,10-05-2021,English,4.5 out of 5 stars12 ratings,615
4,"The Lightning Thief: Percy Jackson, Book 1",Writtenby:RickRiordan,Narratedby:JesseBernstein,10 hrs,13-01-2010,English,4.5 out of 5 stars181 ratings,820


In [5]:
# check for data structure with last 5 values
df1.tail()

,name,author,narrator,time,releasedate,language,stars,price
995,Battle for Loot Lake: An Unofficial Fortnite A...,Writtenby:DevinHunter,Narratedby:RamondeOcampo,2 hrs and 7 mins,18-09-18,English,5 out of 5 stars2 ratings,585
996,Midnight for Charlie Bone,Writtenby:JennyNimmo,Narratedby:SimonRussellBeale,7 hrs and 9 mins,18-12-15,English,Not rated yet,569
997,Winnie-the-Pooh,Writtenby:A.A.Milne,Narratedby:AlanBennett,1 hr and 5 mins,27-01-09,English,Not rated yet,342
998,Midnight on the Moon,Writtenby:MaryPopeOsborne,Narratedby:MaryPopeOsborne,49 mins,13-12-06,English,Not rated yet,301
999,Erak's Ransom,Writtenby:JohnFlanagan,Narratedby:WilliamZappa,12 hrs and 3 mins,04-06-2012,English,Not rated yet,702


In [6]:
# check for data structure with random 5 values
df1.sample(5)

,name,author,narrator,time,releasedate,language,stars,price
583,Thursdays with the Crown,Writtenby:JessicaDayGeorge,Narratedby:GraceAndrews,5 hrs and 38 mins,10-08-2021,English,Not rated yet,600
369,PJ Masks - Vet aquí els Mini Llops!,"Writtenby:eOne,AnaC.AlcainaPerez",Narratedby:RogerSerradell,11 mins,01-07-2022,catalan,Not rated yet,38
462,Ivy & Bean Break the Fossil Record (Book 3),Writtenby:AnnieBarrows,Narratedby:HallieRicardo,1 hr and 8 mins,01-04-2022,English,Not rated yet,500
967,Black Beauty,Writtenby:AnnaSewell,Narratedby:RalphCosham,5 hrs and 21 mins,22-12-11,English,4 out of 5 stars1 rating,668
452,¡Sálvese quien pueda!,Writtenby:JordiSierraiFabra,Narratedby:AnetaFernández,2 hrs and 47 mins,12-01-2021,spanish,Not rated yet,192


In [7]:
# checking general information about data no of rows,columns,non-null values,memory usage,dtype
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         1000 non-null   object
 1   author       1000 non-null   object
 2   narrator     1000 non-null   object
 3   time         1000 non-null   object
 4   releasedate  1000 non-null   object
 5   language     1000 non-null   object
 6   stars        1000 non-null   object
 7   price        1000 non-null   object
dtypes: object(8)
memory usage: 62.6+ KB


#### Working on price column
#### Define 
- Replacing Free with "0" when we use replace on string str.replace() only work on string and for int & float it return nan
- Change dtype to np.float32 for memory optimization

In [8]:
# checking unique values in price if any other values than Free 
df1["price"].unique()

array(['468', '820', '410', '615', '656', '233', '1,256.00', '1,206.00',
       '1,093.00', '467', '836', '1,003.00', '957', '683', '469', '374',
       '702', '501', '284', '575', '668', '305', '1,170.00', '266',
       '1,005.00', '323', '516', '445', '585', '703', '117', '192', '76',
       '821', '200', '706', '904', '679', '112', '531', '586', '1,172.00',
       '949', '322', '569', '398', '352', '164', '234', '303', '32',
       '166', '38', '75', '799', '664', '204', '568', '186', '797', '99',
       '65', '300', '113', '493', '573', '759', '367', '338', '1,382.00',
       '105', '422', '754', '492', '529', '189', '258', '609', '1,328.00',
       '187', '726', '645', '151', '115', '149', '263', '307', '116',
       '225', '959', '452', '36', '265', '653', '515', '227', '351',
       '500', '333', '301', '93', '397', '502', '267', '384', '268',
       '153', '733', '434', '228', '600', '535', '968', '854', '132',
       '334', '401', '866', '402', '420', '417', '190', '251', '335

In [9]:
# Checking rows containing price as 'Free'
df1[df1["price"] == "Free"]

,name,author,narrator,time,releasedate,language,stars,price
983,Ringo & the Yogi,Writtenby:PrasunRoy,Narratedby:SiddhantaPinto,5 hrs and 18 mins,19-11-19,English,4 out of 5 stars4 ratings,Free


In [10]:
# why replaced with "0" when we use replace on string str.replace() only work on string and for int & float it return nan 
df1["price"] = df1["price"].replace("Free","0")

In [11]:
#Change dtype to np.float32 for memory optimization
df1["price"] = df1["price"].str.replace(",","").astype(np.float32)

In [12]:
#check if any missing value added in data
df1[df1["price"].isna()]

,name,author,narrator,time,releasedate,language,stars,price


#### Working on stars column
#### Define 
- function creation to seperate start_rating and rating and new columns added in dataframe
- drop of stars column after two seperate columns for start_rating and rating created
- datatype changed of start_rating column to np.float32 for memory optimization

In [13]:
def star_rating(text):
    try:
        return float(re.split("\s",text)[0])
    except:
        return 0

def rating(text):
    try:
        return int(re.split("\s",text)[4].replace("stars",""))
    except:
        return 0 

In [14]:
df1["star_rating"] = df1["stars"].apply(star_rating)
df1["rating"] = df1["stars"].apply(rating)

In [15]:
df1.drop(columns="stars",inplace=True)

In [16]:
df1["star_rating"] = df1["star_rating"].astype(np.float32)

#### Working on language column
#### Define 
- changing all languages to lower case to create consistency in data format
- datatype changed of language column to category for memory optimization

In [17]:
df1["language"] = df1["language"].str.lower().astype("category")

#### Working on releasedate column
#### Define 
- datatype changed of releasedate column to datetime64[ns] for effective utilization

In [18]:
df1["releasedate"] = pd.to_datetime(df1["releasedate"])

C:\Users\ASUS\AppData\Local\Temp\ipykernel_9864\2862309026.py:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df1["releasedate"] = pd.to_datetime(df1["releasedate"])


In [19]:
#checking ammended details 
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   name         1000 non-null   object        
 1   author       1000 non-null   object        
 2   narrator     1000 non-null   object        
 3   time         1000 non-null   object        
 4   releasedate  1000 non-null   datetime64[ns]
 5   language     1000 non-null   category      
 6   price        1000 non-null   float32       
 7   star_rating  1000 non-null   float32       
 8   rating       1000 non-null   int64         
dtypes: category(1), datetime64[ns](1), float32(2), int64(1), object(4)
memory usage: 56.5+ KB


#### Working on time column
#### Define 
- function creation using regex to change hrs and min into munites

In [20]:
def to_min(text):
    pattern = r"(?:(\d+)\s*hrs?)?\s*(?:and)?\s*(?:(\d+)\s*mins?)?"
    match = re.search(pattern, text)
    if match:
        hours = match.group(1) if match.group(1) else "0"
        minutes = match.group(2) if match.group(2) else "0"
        min1 = int(hours)*60 + int(minutes)
        return min1  

In [21]:
df1["time"] = df1["time"].apply(to_min)

#### Working on narrator,author,name columns
#### Define 
- checking all values in narrator startswith Narratedby: and author with Writtenby: if yes
- replace values with "" null string and change in original data
- random check on whole data found different language
- function creation to translate data in narrator,author,name

In [22]:
print(df1["narrator"].str.startswith("Narratedby:").sum())
print(df1["author"].str.startswith("Writtenby:").sum())

1000
1000


In [23]:
df1["narrator"] = df1["narrator"].str.replace("Narratedby:","")
df1["author"] = df1["author"].str.replace("Writtenby:","")

In [24]:
#random check on whole data found different language
pd.read_csv(r"D:\Jupyter\audible_uncleaned(in).csv")["narrator"][87192]

'Narratedby:小野田英一'

In [132]:
# from deep_translator import GoogleTranslator

# # Translate function
# def translate_name(name):
#     try:
#         return GoogleTranslator(source="auto", target="en").translate(name)
#     except:
#         return name  # Return original if translation fails

In [133]:
#df1["name"] = df1["name"].apply(translate_name)

In [134]:
#df1["author"] = df1["author"].apply(translate_name)

In [135]:
#df1["narrator"] = df1["narrator"].apply(translate_name)

In [139]:
df1.to_csv("cleaned_audio.csv",index=False)

In [179]:
#i["author"].unique()[20:30]

array(['Writtenby:DavPilkey', 'Writtenby:DustiBowling',
       'Writtenby:JulieSykes', 'Writtenby:StellaRobson',
       'Writtenby:JudeWatson', 'Writtenby:FranciscoDíazValladares',
       'Writtenby:Mattel', 'Writtenby:Vegetta777,Willyrex',
       'Writtenby:RoshaniChokshi',
       'Writtenby:SophieSchoenwald,NadineReitz-Illustrator'], dtype=object)

In [180]:
#df1["author"].unique()[20:30]

array(['DavPilkey', 'DustiBowling', 'Juliesykes', 'StellaRobson',
       'JudeWatson', 'FRANCISCODÍZVALADARES', 'Mattel',
       'Vegetta777,Willyrex', 'Roshanichokshi',
       'SophieSchoenwald,NadineReitz-Illustrator'], dtype=object)

In [ ]:
# Some details change found after translating the data 

##### ShannonMessenger > Shannonmesser, DanGutman > Dangutman , RuskinBond > Intodring , JuliaDonaldson > Juladonaldson, 
##### AxelScheffler> Axelscheffler , JulieSykes > Juliesykes , 

In [175]:
df1["author"].unique()[:10]

array(['GeronimoStilton', 'RickRiordan', 'JeffKinney', 'SuzanneCollins',
       'WinterMorgan', 'MaryPopeOsborne', 'PhilipPullman',
       'Shannonmesser', 'ChristopherPaolini', 'Eoincolfer'], dtype=object)

In [ ]:
# so decide to skip translation step

In [25]:
df1

,name,author,narrator,time,releasedate,language,price,star_rating,rating
0,Geronimo Stilton #11 & #12,GeronimoStilton,BillLobely,140,2008-08-04,english,468.0,5.0,34
1,The Burning Maze,RickRiordan,RobbieDaymond,788,2018-05-01,english,820.0,4.5,41
2,The Deep End,JeffKinney,DanRussell,123,2020-11-06,english,410.0,4.5,38
3,Daughter of the Deep,RickRiordan,SoneelaNankani,676,2021-10-05,english,615.0,4.5,12
4,"The Lightning Thief: Percy Jackson, Book 1",RickRiordan,JesseBernstein,600,2010-01-13,english,820.0,4.5,181
...,...,...,...,...,...,...,...,...,...
995,Battle for Loot Lake: An Unofficial Fortnite A...,DevinHunter,RamondeOcampo,127,2018-09-18,english,585.0,5.0,2
996,Midnight for Charlie Bone,JennyNimmo,SimonRussellBeale,429,2015-12-18,english,569.0,0.0,0
997,Winnie-the-Pooh,A.A.Milne,AlanBennett,65,2009-01-27,english,342.0,0.0,0
998,Midnight on the Moon,MaryPopeOsborne,MaryPopeOsborne,49,2006-12-13,english,301.0,0.0,0


In [ ]:
in author & narrator column first name and last name not seperated by white space(validity)
row 35 in names column book name contains book categories (14 Short Stories Bundle for Kids: Adventure, Science Fiction, Aesopâ€™s Fables, Classic Fairy Tales and Christmas Story Books for Children)(validity)
There are no proper names in narrator, e.g., "Narratedby:uncredited".(accuracy)
name columns contains 2 info combined book name and its edition
books written by multiple people Writtenby:EdwardWyke-Smith,VeronicaCossanteli comma seperated data
books Narrated by multiple people comma seperated data

In [94]:
df1["name"].sample(10)

127                             The Hidden Kingdom
391           Avengers - TRE spektakulära äventyr!
793                     Le vaillant petit tailleur
230    I Survived the Sinking of the Titanic, 1912
838                          Ein gefährlicher Pakt
686                        Ein rätselhafter Unfall
16                     Magic Tree House Collection
814                          A Thunder of Monsters
667                               Noticias frescas
864                                The Stolen Show
Name: name, dtype: object

In [98]:
# Now, remove the starnge parts from the "name", "author" and "narrator" columns
df1["name"]     = df1["name"].str.encode('ascii', 'ignore').str.decode('ascii').copy()
df1["author"]   = df1["author"].str.encode('ascii', 'ignore').str.decode('ascii').copy()
df1["narrator"] = df1["narrator"].str.encode('ascii', 'ignore').str.decode('ascii').copy()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   name         1000 non-null   object        
 1   author       1000 non-null   object        
 2   narrator     1000 non-null   object        
 3   time         1000 non-null   int64         
 4   releasedate  1000 non-null   datetime64[ns]
 5   language     1000 non-null   category      
 6   price        1000 non-null   float32       
 7   star_rating  1000 non-null   float32       
 8   rating       1000 non-null   int64         
dtypes: category(1), datetime64[ns](1), float32(2), int64(2), object(3)
memory usage: 56.5+ KB


In [103]:
df1["name"].unique()

array(['Geronimo Stilton #11 & #12', 'The Burning Maze', 'The Deep End',
       'Daughter of the Deep',
       'The Lightning Thief: Percy Jackson, Book 1',
       'The Hunger Games: Special Edition', 'Quest for the Diamond Sword',
       'The Dark Prophecy', 'Merlin Mission Collection',
       'The Tyrants Tomb', "The Titan's Curse: Percy Jackson, Book 3",
       'Magic Tree House Collection: Books 9-16',
       'Magic Tree House Collection: Books 1-8',
       'Magnus Chase and the Ship of the Dead', 'Northern Lights',
       'Geronimo Stilton #13 and #14', 'Magic Tree House Collection',
       'Exile', 'Neverseen', 'The Tower of Nero',
       'Eldest: The Inheritance Cycle, Book 2', 'Artemis Fowl',
       'Geronimo Stilton 22 & 24',
       'Percy Jackson and the Battle of the Labyrinth', 'Winnie-the-Pooh',
       'Magic Tree House Collection: Books 25-32',
       'Geronimo Stilton #20 and #21', 'The Red Pyramid',
       "Mystery of the Griefer's Mark", 'Geronimo Stilton #17',
       

In [106]:
df1[df1["name"].str.encode('ascii', 'ignore').isnull()]

,name,author,narrator,time,releasedate,language,price,star_rating,rating


In [107]:
book_version_pattern = re.compile(r"(\s#?\d+\s?(&|and)?\s?#?(\d+)?)|(:?,?\sBooks?\s\d+\s?-?(\d+)?)")
nan_books_version = df1[df1["name"].str.replace(book_version_pattern, "").isnull()]

In [109]:
# remove the versions from the "name"
book_version_pattern = re.compile(r"(\s#?\d+\s?(&|and)?\s?#?(\d+)?)|(:?,?\sBooks?\s\d+\s?-?(\d+)?)")
df1["name"] = df1["name"].str.replace(book_version_pattern, "").copy()

In [114]:
# split those values which have 2 or more author or narrator
df1["author"] = df1["author"].apply(lambda x: x.split(",") if "," in x else [x])
df1["narrator"] = df1["narrator"].apply(lambda x: x if pd.isna(x) else x.split(",") if "," in x else [x])

In [116]:
df1.isnull().sum()

name           0
author         0
narrator       0
time           0
releasedate    0
language       0
price          0
star_rating    0
rating         0
dtype: int64

In [118]:
# remove the more informations from the "author" col because most of the values 
# has no such info. Also, while modifying this col before, some blank strings
# (e.g. record no. 77) created. That strings will be removed here. Also, here
# will separate the first name and the last names. Previously those are not
# separated with a white space.

def separate_first_last_name(text):
    # create the regex pattern
    pattern = re.compile(r"[A-Z]+[a-z]+")
    # join the first & last name with a white space and return it
    return " ".join(re.findall(pattern, text))

def remove_more_info(text_list: list):
    if type(text_list) != list and pd.isna(text_list):
        return text_list

    # create the re pattern to remove the more info
    pattern = re.compile(r"-\w+")
    result = []

    # loop through the each values
    for item in text_list:
        # remove the empty string
        if len(item) != 0:
            # remove the more info
            re_result = re.sub(pattern, "", item)
            # separate the first and the last name
            result.append(separate_first_last_name(re_result))
    
    # if the result list is empty, then return NaN or return as it is
    return np.nan if len(result) <= 0 else result

In [120]:
df1["author"]   = df1["author"].apply(lambda x: remove_more_info(x)).copy()
df1["narrator"] = df1["narrator"].apply(lambda x: remove_more_info(x)).copy()

In [122]:
df1.head()

,name,author,narrator,time,releasedate,language,price,star_rating,rating
0,Geronimo Stilton,[Geronimo Stilton],[Bill Lobely],140,2008-08-04,english,468.0,5.0,34
1,The Burning Maze,[Rick Riordan],[Robbie Daymond],788,2018-05-01,english,820.0,4.5,41
2,The Deep End,[Jeff Kinney],[Dan Russell],123,2020-11-06,english,410.0,4.5,38
3,Daughter of the Deep,[Rick Riordan],[Soneela Nankani],676,2021-10-05,english,615.0,4.5,12
4,The Lightning Thief: Percy Jackson,[Rick Riordan],[Jesse Bernstein],600,2010-01-13,english,820.0,4.5,181


In [123]:
df1.to_pickle("audiable_cleaned.pickle")

In [124]:
pd.read_pickle("audiable_cleaned.pickle").head()

,name,author,narrator,time,releasedate,language,price,star_rating,rating
0,Geronimo Stilton,[Geronimo Stilton],[Bill Lobely],140,2008-08-04,english,468.0,5.0,34
1,The Burning Maze,[Rick Riordan],[Robbie Daymond],788,2018-05-01,english,820.0,4.5,41
2,The Deep End,[Jeff Kinney],[Dan Russell],123,2020-11-06,english,410.0,4.5,38
3,Daughter of the Deep,[Rick Riordan],[Soneela Nankani],676,2021-10-05,english,615.0,4.5,12
4,The Lightning Thief: Percy Jackson,[Rick Riordan],[Jesse Bernstein],600,2010-01-13,english,820.0,4.5,181
